<h1 style="text-align:center">Data Science and Machine Learning Capstone Project</h1>
<img style="float:right" src="https://prod-edxapp.edx-cdn.org/static/edx.org/images/logo.790c9a5340cb.png">
<p style="text-align:center">IBM: DS0720EN</p>
<p style="text-align:center">Question 3 of 4</p>

1. [Problem Statement](#problem)
2. [Question 3](#question)
3. [Data Cleaning and Standardization](#wrangling)
4. [Analyzing and Visualizing](#analysis)
5. [Concluding Remarks](#conclusion)

<a id="problem"></a>
# Problem Statement
---

The people of New York use the 311 system to report complaints about the non-emergency problems to local authorities. Various agencies in New York are assigned these problems. The Department of Housing Preservation and Development of New York City is the agency that processes 311 complaints that are related to housing and buildings.

In the last few years, the number of 311 complaints coming to the Department of Housing Preservation and Development has increased significantly. Although these complaints are not necessarily urgent, the large volume of complaints and the sudden increase is impacting the overall efficiency of operations of the agency.

Therefore, the Department of Housing Preservation and Development has approached your organization to help them manage the large volume of 311 complaints they are receiving every year.

The agency needs answers to several questions. The answers to those questions must be supported by data and analytics. These are their  questions:

<a id="question"></a>
# Question 3
---

Does the Complaint Type that you identified in response to Question 1 have an obvious relationship with any particular characteristic or characteristic of the Houses?

## Approach
Determine how to link the 311 data with the PLUTO data then identify whether or not there are any correlations between the HEAT/HOT WATER complaints from Question 1 to the PLUTO house information.

## Load Data
Separately from this notebook:

The [New York 311](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9) data was loaded by [SODA](https://data.cityofnewyork.us/resource/fhrw-4uyv.csv?$limit=100000000&Agency=HPD&$select=created_date,unique_key,complaint_type,incident_zip,incident_address,street_name,address_type,city,resolution_description,borough,latitude,longitude,closed_date,location_type,status) into a Pandas DataFrame then saved to a pickle file.

The [New York PLUTO](https://data.cityofnewyork.us/City-Government/Primary-Land-Use-Tax-Lot-Output-PLUTO-/xuk2-nczf) data was downloaded.  The instructions at ( Course / 1. Project Challenge Details and Setup / Datasets Used in this Course / Datasets ) said "Use only the part that is specific to the borough that you are interested in based on your analysis."  My answer for Question 2 suggested the borough with the biggest HEAT/HOT WATER problem was BRONX.  For that reason, only the BX_18v1.csv file was loaded into a Pandas DataFrame then saved to a pickle file.

In [29]:
import pandas as pd
files_path = 'C:\\Users\\It_Co\\Documents\\DataScience\\Capstone\\' #local
#files_path = './' #IBM Cloud / Watson Studio

b311 = pd.read_pickle(files_path + 'ny311full.pkl')

#bPlu = pd.read_csv(files_path + 'BX_18v1.csv', usecols=['Address','BldgArea','BldgDepth','BuiltFAR','CommFAR','FacilFAR','Lot','LotArea','LotDepth','NumBldgs','NumFloors','OfficeArea','ResArea','ResidFAR','RetailArea','YearBuilt','YearAlter1','ZipCode','YCoord','XCoord'])
#bPlu.to_pickle(files_path + 'BX_18v1.pkl')
bPlu = pd.read_pickle(files_path + 'BX_18v1.pkl')

print("NY 311 shape %s" % (b311.shape,))
print("BRONX PLUTO shape %s" % (bPlu.shape,))

NY 311 shape (5862383, 15)
BRONX PLUTO shape (89854, 20)


<a id="wrangling"></a>
# Data Cleaning and Standardization
---

Observations with missing or malformed data elements will need to be corrected or removed.  The 311 and the PLUTO data sets will need to be "joined" together by the common "address" element, which means the addresses will need to be standardized to a consistent layout to allow the addresses to be compared consistently.

## NY 311

### General

In [2]:
#Normalize relevant strings to uppercase so different casing won't appear as separate values.
b311['incident_address'] = b311['incident_address'].str.upper()
b311['city'] = b311['city'].str.upper()
b311['borough'] = b311['borough'].str.upper()
#We only care if it is the combined "heating and hot water" from Question 1, or some other type of complaint.
b311.loc[b311[b311["complaint_type"].isin(["HEAT/HOT WATER","HEATING"])==True].index,'complaint_type'] = "HEAT/HOT WATER"
b311.loc[b311[b311["complaint_type"].isin(["HEAT/HOT WATER","HEATING"])==False].index,'complaint_type'] = "OTHER"

In [3]:
#Print some initial information for comparison during later steps.
print("shape %s" % str(b311.shape))
print("--nulls below--")
print(b311.isnull().sum())
print("--types below--")
print(b311.dtypes)
b311.head()

shape (5862383, 15)
--nulls below--
created_date                   0
unique_key                     0
complaint_type                 0
incident_zip               80611
incident_address           52831
street_name                52831
address_type               84775
city                       80210
resolution_description     13194
borough                        0
latitude                   80585
longitude                  80585
closed_date               123122
location_type                  0
status                         0
dtype: int64
--types below--
created_date               object
unique_key                 object
complaint_type             object
incident_zip              float64
incident_address           object
street_name                object
address_type               object
city                       object
resolution_description     object
borough                    object
latitude                  float64
longitude                 float64
closed_date                objec

created_date unique_key  complaint_type  incident_zip  \
0  2019-09-28T20:31:44.000   43918968           OTHER       11435.0   
1  2019-09-28T20:16:49.000   43917365           OTHER       11223.0   
2  2019-09-28T16:10:10.000   43917912  HEAT/HOT WATER       11233.0   
3  2019-09-28T14:50:47.000   43918939           OTHER       10456.0   
4  2019-09-28T09:28:25.000   43916893           OTHER       11205.0   

           incident_address        street_name address_type      city  \
0  141-40 PERSHING CRESCENT  PERSHING CRESCENT      ADDRESS   JAMAICA   
1     1702 WEST    1 STREET   WEST    1 STREET      ADDRESS  BROOKLYN   
2        1711 FULTON STREET      FULTON STREET      ADDRESS  BROOKLYN   
3      881 CAULDWELL AVENUE   CAULDWELL AVENUE      ADDRESS     BRONX   
4        196 CLINTON AVENUE     CLINTON AVENUE      ADDRESS  BROOKLYN   

                              resolution_description   borough   latitude  \
0  The following complaint conditions are still o...    QUEENS  40.712047   
1  The following complaint conditions are still o...  BROOKLYN  40.606232   
2  The complaint you filed is a duplicate of a co...  BROOKLYN  40.679340   
3  The following complaint conditions are still o...     BRONX  40.822361   
4  The following complaint conditions are still o...  BROOKLYN  40.692252   

   longitude closed_date         location_type status  
0 -73.815983         NaN  RESIDENTIAL BUILDING   Open  
1 -73.974553         NaN  RESIDENTIAL BUILDING   Open  
2 -73.930435         NaN  RESIDENTIAL BUILDING   Open  
3 -73.907267         NaN  RESIDENTIAL BUILDING   Open  
4 -73.968649         NaN  RESIDENTIAL BUILDING   Open

### Standardize Borough
Leveraging findings found while standardizing during Question 2.

In [4]:
b311['borough'].value_counts()

BROOKLYN         1692795
BRONX            1566246
MANHATTAN        1020161
UNSPECIFIED       873226
QUEENS            624323
STATEN ISLAND      85632
Name: borough, dtype: int64

In [5]:
import numpy as np
#Correct rows where borough was entered in the city column with "UNSPECIFIED" in the borough column.
five_boroughs = ["BROOKLYN","BRONX","MANHATTAN","QUEENS","STATEN ISLAND"]
which_rows_to_adjust = b311[(b311["borough"]=='UNSPECIFIED')&b311["city"].isin(five_boroughs)].index
b311.loc[which_rows_to_adjust,'borough']=b311.loc[which_rows_to_adjust,'city']
b311.loc[which_rows_to_adjust,'city']=np.nan
#Drop a few rows of ambiguous data.
b311.drop(b311[(b311["borough"]=='MANHATTAN')&(b311["city"]=='BRONX')].index, axis=0, inplace=True)
b311.reset_index(drop=True, inplace=True)
#Fill in UNSPECIFIED borough when city was entered as NEW YORK.
which_rows_to_adjust = b311[(b311["borough"]=='UNSPECIFIED')&(b311["city"]=='NEW YORK')].index
b311.loc[which_rows_to_adjust,'borough']="MANHATTAN"
b311.loc[which_rows_to_adjust,'city']=np.nan
#Although the city for most of the "NEW YORK" ones are the only ones that technically got the "city" column valued correctly,
#since every other row uses city as "neighborhood":  Standardize these.
which_rows_to_adjust = b311[(b311["city"]=='NEW YORK')].index
b311.loc[which_rows_to_adjust,'city']=np.nan
#Any still unspecified boroughs with a value in "city" are in the Queens borough.  The "city" is actually a "neighborhood".
queens_neighborhoods = b311[(b311['borough']=='UNSPECIFIED')&(b311['city'].isnull()==False)]['city'].unique()
#Standardize borough for Queens neighborhoods.
which_rows_to_adjust = b311[(b311["borough"]=='UNSPECIFIED')&b311["city"].isin(queens_neighborhoods)].index
b311.loc[which_rows_to_adjust,'borough']="QUEENS"
#Null the borough if it still shows up as unspecified borough as there is no other information from which to derive it.
which_rows_to_adjust = b311[(b311["borough"]=='UNSPECIFIED')&b311["city"].isnull()].index
b311.loc[which_rows_to_adjust,'borough']=np.nan

In [6]:
b311['borough'].value_counts()

BROOKLYN         1988158
BRONX            1816914
MANHATTAN        1175218
QUEENS            728676
STATEN ISLAND      99998
Name: borough, dtype: int64

### Filter NY 311 data by borough to only include BRONX

In [7]:
b311.drop(b311[(b311["borough"]!='BRONX')].index, axis=0, inplace=True)
b311.reset_index(drop=True, inplace=True)
b311['borough'].value_counts()

BRONX    1816914
Name: borough, dtype: int64

### Remove unnecessary columns

In [8]:
#Remove columns deemed unnecessary for this question.
b311.drop(['created_date','street_name','address_type','city','resolution_description','borough','closed_date','location_type','status','unique_key'], axis=1, inplace=True)
print(b311.shape)
print(b311.isnull().sum())
b311.head()

(1816914, 5)
complaint_type         0
incident_zip        8083
incident_address       1
latitude            8083
longitude           8083
dtype: int64


complaint_type  incident_zip       incident_address   latitude  longitude
0           OTHER       10456.0   881 CAULDWELL AVENUE  40.822361 -73.907267
1  HEAT/HOT WATER       10457.0          4487 3 AVENUE  40.853236 -73.891742
2           OTHER       10452.0  133 CLARKE PLACE EAST  40.837417 -73.916512
3           OTHER       10453.0    2076 CRESTON AVENUE  40.853689 -73.903955
4           OTHER       10471.0          6035 BROADWAY  40.891859 -73.897216

In [9]:
# Drop the one observation with the missing address as there will be no way to tie it to any PLUTO data.
b311.dropna(subset=['incident_address'], axis=0, inplace=True)
b311.reset_index(drop=True, inplace=True)
print(b311.isnull().sum())
b311['incident_address'].value_counts().head()

complaint_type         0
incident_zip        8082
incident_address       0
latitude            8082
longitude           8082
dtype: int64


1025 BOYNTON AVENUE        9854
3810 BAILEY AVENUE         7171
750 GRAND CONCOURSE        4412
888 GRAND CONCOURSE        4271
3555 BRUCKNER BOULEVARD    4076
Name: incident_address, dtype: int64

## BRONX PLUTO

In [14]:
print("shape %s" % str(bPlu.shape))
print("---isnull follows---")
print(bPlu.isnull().sum())
bPlu.head()

shape (89854, 21)
---isnull follows---
Lot              0
ZipCode        329
Address         69
BldgClass       66
LotArea          0
BldgArea         0
ResArea          0
OfficeArea       0
RetailArea       0
NumBldgs         0
NumFloors        0
LotDepth         0
BldgDepth        0
YearBuilt        0
YearAlter1       0
BuiltFAR         0
ResidFAR         0
CommFAR          0
FacilFAR         0
XCoord        3259
YCoord        3259
dtype: int64


Lot  ZipCode                 Address BldgClass  LotArea  BldgArea  ResArea  \
0    1  10454.0  122 BRUCKNER BOULEVARD        Z9    15000         0        0   
1    4  10454.0  126 BRUCKNER BOULEVARD        G5    13770       752        0   
2   10  10454.0  138 BRUCKNER BOULEVARD        F5    35000     39375        0   
3   17  10454.0  144 BRUCKNER BOULEVARD        C1     2500     12500    12500   
4   18  10454.0  148 BRUCKNER BOULEVARD        C7     1875      8595     6876   

   OfficeArea  RetailArea  NumBldgs  ...  LotDepth  BldgDepth  YearBuilt  \
0           0           0         1  ...     200.0        0.0          0   
1         272           0         2  ...     100.0       16.0       1931   
2           0           0         1  ...     200.0      200.0       1931   
3           0           0         1  ...     100.0       85.0       1931   
4           0        1719         1  ...      75.0       70.0       1920   

   YearAlter1  BuiltFAR  ResidFAR  CommFAR  FacilFAR     XCoord    YCoord  
0           0      0.00      6.02      5.0       6.5  1005957.0  232162.0  
1        1994      0.05      6.02      5.0       6.5  1006076.0  232156.0  
2           0      1.13      6.02      5.0       6.5  1006187.0  232036.0  
3        2001      5.00      6.02      5.0       6.5  1006299.0  232033.0  
4        2009      4.58      6.02      5.0       6.5  1006363.0  232040.0  

[5 rows x 21 columns]

### General

In [16]:
bPlu.dtypes

Lot             int64
ZipCode       float64
Address        object
BldgClass      object
LotArea         int64
BldgArea        int64
ResArea         int64
OfficeArea      int64
RetailArea      int64
NumBldgs        int64
NumFloors     float64
LotDepth      float64
BldgDepth     float64
YearBuilt       int64
YearAlter1      int64
BuiltFAR      float64
ResidFAR      float64
CommFAR       float64
FacilFAR      float64
XCoord        float64
YCoord        float64
dtype: object

In [17]:
#Normalize relevant strings to uppercase so different casing won't appear as separate values.
bPlu['Address'] = bPlu['Address'].str.upper()

In [18]:
# Drop the observations with missing address as there will be no way to tie it to any 311 data.
bPlu.dropna(subset=['Address'], axis=0, inplace=True)
bPlu.reset_index(drop=True, inplace=True)

## Standardization of Addresses

In [19]:
print("BRONX 311 unique addresses: %s" % b311['incident_address'].unique().size)
print("BRONX PLUTO unique addresses: %s" % bPlu['Address'].unique().size)

BRONX 311 unique addresses: 29216
BRONX PLUTO unique addresses: 87017


In [20]:
#Determine how much overlap.  Ideally all 29K BRONX 311 addresses will be represented in the PLUTO set.

In [21]:
def WhichAddressesNotInPluto():
    complaints = set(b311['incident_address'].unique())
    pluto = set(bPlu['Address'].unique())
    #Determine which 311 addresses were not found in PLUTO to gain insight as to why.
    differences = complaints.difference(pluto)
    print("Records not in PLUTO: %s.  Percent: %s" % (len(differences), "{:.2%}".format(len(differences) / len(complaints))))
    print("---Top 5---")
    print(b311[b311['incident_address'].isin(differences)]['incident_address'].value_counts().head())

In [22]:
WhichAddressesNotInPluto()

Records not in PLUTO: 6753.  Percent: 23.11%
---Top 5---
2090 EAST TREMONT AVENUE          3940
266 BEDFORD PARK BOULEVARD        2594
1425 DR M L KING JR BOULEVARD     2534
116 EAST MOSHOLU PARKWAY SOUTH    2312
1259 CLAY AVENUE                  2297
Name: incident_address, dtype: int64


In [96]:
#Try to find
bPlu[(bPlu['Address'].str.startswith('20'))&(bPlu['Address'].str.contains('TREMONT'))]['Address'].sort_values()

11574     200 EAST TREMONT AVENUE
13304     200 WEST TREMONT AVENUE
38801    2000 EAST TREMONT AVENUE
41064    2015 EAST TREMONT AVENUE
38802    2020 EAST TREMONT AVENUE
38803    2040 EAST TREMONT AVENUE
Name: Address, dtype: object

In [89]:
#Almost 4000 complaints at 2090 EAST TREMONT AVENUE, but the address isn't in PLUTO?
#Google maps shows more than one building with this address, and those buildings also have other addresses in the same building.
#See if there is a way to determine the real address by looking at latitude and longitude.

#locations = b311[b311['incident_address']=="2090 EAST TREMONT AVENUE"][["latitude","longitude"]].dropna(axis=0,inplace=False).drop_duplicates()
#locations
# 40.84196, -73.857725
# 40.84196, -73.857736
top = 40.846
bottom = 40.836
left = -73.862
right = -73.852
locations = b311[(b311['latitude'].ge(bottom))&(b311['latitude'].le(top))&(b311['longitude'].ge(left))&(b311['longitude'].le(right))][["latitude","longitude","incident_address"]].dropna(axis=0,inplace=False).drop_duplicates()
#locations

In [90]:
import folium

In [91]:
area_map = folium.Map(location=[40.84196, -73.857725], zoom_start=18)
markers = folium.map.FeatureGroup()
for lat, lng, pop in zip(locations['latitude'], locations['longitude'], locations['incident_address']):
    markers.add_child(
        folium.features.RegularPolygonMarker([lat,lng],radius=5,color='red',fill=True,fill_color='orange',fill_opacity=1.0,popup=pop)
    )
area_map.add_child(markers)
area_map

# Continue here
Make whatever conclusions necessary using the map.  
Can things be mapped using the coordinates in the PLUTO file?
Is there an entry in the PLUTO file with a grossly mis-spelled version of TREMONT AVENUE?

## Using each data set to fill in missing items in the other.

<a id="analysis"></a>
# Analyzing and Visualizing
---

In [ ]:
b311.complaint_type.isnull().sum()

<p style="color:Red;">None of the complaint types are null.</p>

In [ ]:
b311['complaint_type'].describe()

<p style="color:Red;">HEAT/HOT WATER is the most common of the 29 unique complaint types, but closer examination is necessary to make a final answer to the question.</p>

In [ ]:
unique_types = b311['complaint_type'].unique()
unique_types.sort()
unique_types

<p style="color:Red;">Some of these appear to be duplicate ways to represent the same thing.  The Open Data Page does not provide any clarification of these values, in case for example, water leaks should be lumped in with plumbing.  In a more real situation we could ask for such clarification.  If that was not available an analysis of whether apparent duplicates come from different Open_Data_Channel_Type could help formulate the best guess why there are apparent duplicates.</p>

In [ ]:
#Although we may need dummy values for later questions, for now just normalize the data in-place.
#Using best guess interpretation about which complaint types are equivalent to each other.
b311['complaint_type'].replace('Appliance', 'APPLIANCE', inplace = True)
b311['complaint_type'].replace('GENERAL CONSTRUCTION', 'CONSTRUCTION', inplace = True)
b311['complaint_type'].replace('General', 'GENERAL', inplace = True)
b311['complaint_type'].replace('HEATING', 'HEAT/HOT WATER', inplace = True)
b311['complaint_type'].replace('Outside Building', 'OUTSIDE BUILDING', inplace = True)
b311['complaint_type'].replace('PAINT - PLASTER', 'PAINT/PLASTER', inplace = True)
b311['complaint_type'].replace('Plumbing', 'PLUMBING', inplace = True)
b311['complaint_type'].replace('Safety', 'SAFETY', inplace = True)
b311['complaint_type'].replace('Unsanitary Condition', 'UNSANITARY CONDITION', inplace = True)

In [ ]:
print(b311['complaint_type'].describe())
unique_types = b311['complaint_type'].unique()
unique_types.sort()
unique_types

<p style="color:Red;">Twenty complaint types after consolidating obvious duplicates.

In [ ]:
print (1 - 2040461.00 / 5862383.00) # taking values from the "describe" statement above.

<p style="color:Red;">HEAT/HOT WATER represents almost 2/3 of the complaints.</p>

In [ ]:
print(b311["complaint_type"].value_counts())    

<p style="color:Red;">The next two or three most common complaints combined happen less.</p>

<p style="color:Red;">Let's take a more visual look, just to double check.</p>

In [ ]:
#Plot a bar chart to show how many of each type of complaint.
%matplotlib inline
import matplotlib.pyplot as plt
totals = pd.DataFrame(b311['complaint_type'].value_counts())
totals.plot(kind='bar', figsize=(10,6), rot=90)
plt.xlabel("Complaint Type")
plt.ylabel("Number of Complaints")
plt.title("Number of complaints by type")
plt.annotate('', xycoords='data', xy=(1, 2000000), xytext=(5, 1500000), arrowprops=dict(arrowstyle='->', connectionstyle='arc3', color='r', lw=2))
plt.show()

<p style="color:Red;">Clearly, nothing else comes close to HEAT/HOT WATER over the entire data set.</p>

<p style="color:Red;">Perhaps the top complaint changes over time and HEAT/HOT WATER is only the top one in aggregate?  Let's see if that has been the case by also looking at totals for each individual year.</p>

In [ ]:
#Add Year column.
def strleft(s):
    return s[0:4]
b311['Year'] = b311['created_date'].apply(strleft)

In [ ]:
#Determine unique list of years.
unique_years = b311['Year'].unique()
unique_years.sort()

In [ ]:
#Calculate yearly totals
import numpy as np
yearly_totals = pd.DataFrame(index=unique_types, columns=unique_years)
for y in unique_years:
    values_this_year = b311[b311['Year']==y]['complaint_type'].value_counts()
    for i in values_this_year.index:
        yearly_totals.at[i,y] = values_this_year[i]
yearly_totals.replace(np.nan, 0, inplace = True)
yearly_totals = yearly_totals.transpose()

In [ ]:
#Plot the number of complaints by type and year.
yearly_totals.plot(kind="line")
plt.title('Complaints Over the Years')
plt.ylabel('Number of Complaints')
plt.xlabel('Year')
plt.legend(bbox_to_anchor=(1.05,1))
plt.show()

<p style="color:Red;">The top complaint type consistently every year is HEAT/HOT WATER.</p>

<p style="color:Red;">All the totals drop off at the end.  This is because the data set is updated every day, but the final year 2019 is still in progress with almost three months to go.</p>

<a id="conclusion"></a>
# Concluding Remarks
---

The HEAT/HOT WATER (including HEATING) complaint type identified in Question 1 as the most prevalent complaint type has (or does not have any????) an obvious relationship with the XXX house characteristics: